In [1]:
import matplotlib.pyplot as plt
import numpy as np
import ase.db
import json

from ase.visualize import view
from sklearn.kernel_ridge import KernelRidge as KRR
from sklearn.neighbors import KNeighborsRegressor as KNN

In [2]:
# Connect to database
con = ase.db.connect("catapp.db")

In [3]:
reactions = {}

i = 0
for row in con.select():
    try:
        reaction = {}
        reaction['reactant_a'] = row.a
        reaction['reactant_b'] = row.b
        reaction['product_ab'] = row.ab
        reaction['reactant_surface'] = row.surface
        reaction['reactant_facet'] = row.facet
        try:
            reaction['site'] = row.site
        except AttributeError:
            reaction['site'] = 'None'
        reaction['reaction_energy'] = row.er
        reaction['activation_energy'] = row.ea
        reaction['dft_functional'] = row.xc
        reactions[i] = reaction
        i += 1
    except AttributeError:
        pass

with open('../model/reactions_catapp.json', 'w') as f:
    json.dump(reactions, f)

print('All reactions read into file')

All reactions read into file
